# How to use
Run all and enter data when prompted

In [1]:
from torch.cuda import graph

# Get documents
documents = []

for i in range(int(input("Enter the number of documents for the AI to understand"))):
    documents.append(input("Copy the document here").replace("\n\n", ""))

documents

['Will Wood is an American musician, singer-songwriter, and comedian.[4][5] Wood has released four studio albums; Everything Is a Lot (2015), SELF-iSH (2016), The Normal Album (2020),[6] and "In case I make it," (2022). The former two were released as Will Wood and the Tapeworms, Wood\'s prior band name. He has additionally released two live albums and a soundtrack.\nStyleWood\'s piano-led style often changes from one song to the next, drawing influence from folk, pop, jazz, rock and roll, latin music, doo wop, and klezmer. He is also known for his unconventional use of tenor and baritone ukulele. When recording in studio or performing live with his band, "the Tapeworms", he has mostly been accompanied by Mike Bottiglieri on guitar, Matt Berger on alto saxophone, Mario Conte on drums, Vater Boris on bass, and Rob Schaefer on trumpet.[1][7] The band is known for high energy live performances.[8]']

In [2]:
question = input("Enter your question: ") # Can be skipped if only focusing on GraphGen

KeyboardInterrupt: Interrupted by user

In [2]:
graph_gen = None

In [3]:
from GraphGen.graph_gen_integration import GraphGen
# Parse documents
triples = []
chunks = []

if graph_gen is None:
    graph_gen = GraphGen([], keyword_reward=1.05)


for idx, document in enumerate(documents):
    print(f"==================== Process document {idx+1}/{len(documents)} ====================")
    graph_gen.reset_model(triples, idx)
    triples += graph_gen.generate(document)
    chunks.append(graph_gen.split_chunks(document))
    
print("ACTUALLY DONE!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 2060. Max memory: 6.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


==================== Process document 1/1 ====================
PROCESSING (1/5)
Phase 1 text chunk "Will Wood is an American musician, singer-songwrit...ms and a soundtrack."
Phase 1 context "None...None"
---
Phase 1 model start
Phase 2 model generation done

Will_Wood<unused1>genre<unused2>Indie_rock
Will_Wood<unused1>background<unused2>"solo_singer"
Will_Wood<unused1>birthPlace<unused2>United_States
Will_Wood<unused1>recordLabel<unused2>Fat_Cat_Records
Will_Wood<unused1>associatedBand/associatedMusicalArtist<unused2>The_Tapeworms
Will_Wood<unused1>activeYearsStartYear<unused2>2005
Will_Wood<unused1>recordLabel<unused2>Epitaph_Records
Will_Wood<unused1>recordLabel<unused2>Sub Pop
Phase 3 parsing done
chunk DONE
PROCESSING (2/5)
Phase 1 text chunk "He has additionally released two live albums and a...ms and a soundtrack."
Phase 1 context "<unused0>Will_Wood<unused1>ass...nused2>"solo_singer""
---
Phase 1 model start
Phase 2 model generation done

Will_Wood<unused1>recordLabel<unused2>F

In [4]:
triples

[Triple(subject='Will_Wood', predicate='genre', object='Indie_rock', id=0, document_id=0),
 Triple(subject='Will_Wood', predicate='background', object='"solo_singer"', id=0, document_id=0),
 Triple(subject='Will_Wood', predicate='birthPlace', object='United_States', id=0, document_id=0),
 Triple(subject='Will_Wood', predicate='recordLabel', object='Fat_Cat_Records', id=0, document_id=0),
 Triple(subject='Will_Wood', predicate='associatedBand/associatedMusicalArtist', object='The_Tapeworms', id=0, document_id=0),
 Triple(subject='Will_Wood', predicate='activeYearsStartYear', object='2005', id=0, document_id=0),
 Triple(subject='Will_Wood', predicate='recordLabel', object='Epitaph_Records', id=0, document_id=0),
 Triple(subject='Will_Wood', predicate='recordLabel', object='Sub Pop', id=0, document_id=0),
 Triple(subject='Will_Wood', predicate='instrument', object='Piano', id=2, document_id=0),
 Triple(subject='Will_Wood', predicate='stylisticOrigin', object='Jazz', id=2, document_id=0),


In [5]:
from pyvis.network import Network
from GraphGen.graph_gen_integration import Triple
from GraphGen import graph_gen_integration

net = Network(bgcolor="#222222", font_color="white", notebook=True, directed=True)
net2 = Network(bgcolor="#222222", font_color="white", notebook=False, directed=True)


# Parse rdf_strings

def add_triples(rdf: Triple, _color: str):
    net.add_node(rdf.subject, color=_color)
    net.add_node(rdf.object, color=_color)
    # if not any(edge['from'] == rdf.subject and edge['to'] == rdf.object and edge['title'] == rdf.predicate for edge in net.edges): # should be deprecated later
    net.add_edge(rdf.subject, rdf.object, title=rdf.predicate, color=_color)
    
    net2.add_node(rdf.subject, color=_color)
    net2.add_node(rdf.object, color=_color)
    # if not any(edge['from'] == rdf.subject and edge['to'] == rdf.object and edge['title'] == rdf.predicate for edge in net.edges): # should be deprecated later
    net2.add_edge(rdf.subject, rdf.object, title=rdf.predicate, color=_color)


import random

r = lambda: random.randint(0, 255)
net.toggle_physics(True)

colors = {}

for idx, triple in enumerate(triples):
    if triple.document_id in colors:
        document = colors[triple.document_id]
        if triple.id in document:
            color = document[triple.id]
        else:
            color = '#%02X%02X%02X' % (r(), r(), r())
            colors[triple.document_id][triple.id] = color
    else:
        colors[triple.document_id] = {} 
        color = '#%02X%02X%02X' % (r(), r(), r())
        colors[triple.document_id][triple.id] = color

    add_triples(triple, color)
    # net.show(f"{idx}.html", notebook=False)
net.show("graph.html", notebook=True)
# net2.show("graph2.html", notebook=False) # For visualization outside notebook

graph.html


In [6]:
# Workaround for now. Backtracking to source
def get_chunk(_hex_code):
    _hex_code = _hex_code.upper()
    print(_hex_code)
    possible_docs = []
    for _document in colors:
        for _chunk in colors[_document]:
            if colors[_document][_chunk] == _hex_code:
                possible_docs.append([_document, _chunk])
                
    if len(possible_docs) == 0:
        return None
    
    # print(possible_docs)
    
    docs = []
    
    for doc in possible_docs:
       docs.append(graph_gen.split_chunks(graph_gen.preprocess_document(documents[doc[0]]))[doc[1]])
    
    return docs

In [14]:
graph_gen.free_model()
del graph_gen

In [15]:
import torch, gc
# graph_gen.free_model()
# del graph_gen
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from QueryGen.query_gen_integration import QueryGen
query_gen = QueryGen(triples)

In [10]:
# question = input("Enter your question: ")
query = query_gen.generate(question)
query

'SELECT DISTINCT ?uri WHERE { <Los_Angeles_Dodgers> <locationCity> ?uri }'

In [39]:
from rdflib import Graph, Literal, RDF, URIRef
g = Graph()
for triple in triples:
    g.add((URIRef(triple.subject), URIRef(triple.predicate), URIRef(triple.object)))

"Nationwide" does not look like a valid URI, trying to serialize this will break.
"Scholars' Bowl" does not look like a valid URI, trying to serialize this will break.
"Scholars' Bowl", "Academic Bowl", "Academic Team", "Academic Challenge", "Scholastic Bowl", "Primary School Quiz Bowl", "Middle School Quiz Bowl", "High School Quiz Bowl", "University Quiz Bowl" does not look like a valid URI, trying to serialize this will break.
"Nationwide" does not look like a valid URI, trying to serialize this will break.
"Unlimited" does not look like a valid URI, trying to serialize this will break.
"Unlimited" does not look like a valid URI, trying to serialize this will break.
"Single-elimination tournament" does not look like a valid URI, trying to serialize this will break.
"4 per team" does not look like a valid URI, trying to serialize this will break.
"Buzzer" does not look like a valid URI, trying to serialize this will break.
"1953-09-28" does not look like a valid URI, trying to seriali

In [40]:
# Apply the query to the graph and iterate through results
r_r = []
for r in g.query(query):
    print(r[0])
    r_r.append(str(r[0]))
r_r

[]

In [45]:
query_gen.free_model()
del query_gen
gc.collect()
torch.cuda.empty_cache()

NameError: name 'query_gen' is not defined

In [42]:
from AnswerGen.answer_gen_integration import AnswerGen
answer_gen = AnswerGen()

answer = answer_gen.generate(question, str(r_r))
answer

==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 2060. Max memory: 6.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


'The quizbowl is the sport of the quiz bowl.<eos>'

In [46]:
answer_gen.free_model()
del answer_gen
gc.collect()
torch.cuda.empty_cache()

In [43]:
print(answer.replace("<eos>", ""))

The quizbowl is the sport of the quiz bowl.


In [ ]:
graph_gen = GraphGen([], keyword_reward=1.1)
graph_gen.reset_model([], 0)

In [8]:
# CHUNK LOOKUP
hex_code = input("Enter hex code for chunk lookup")
chunks = get_chunk(hex_code)
if chunks is None:
    print("No related chunk found. Try again")
elif len(chunks) == 1:
    print("CHUNK: ")
    print(chunks[0])
elif len(chunks) > 1:
    print(f"{len(chunks)} chunks somehow got assigned the same color. Find them below")
    for chunk in chunks:
        print(chunk)

#D55923
CHUNK: 
Will Wood is an American musician, singer-songwriter, and comedian. [4][5] Wood has released four studio albums; Everything Is a Lot (2015), SELF-iSH (2016), The Normal Album (2020),[6] and "In case I make it," (2022). The former two were released as Will Wood and the Tapeworms, Wood's prior band name. He has additionally released two live albums and a soundtrack.


In [50]:
colors

{0: {0: '#2B6E1E', 1: '#A29881', 2: '#74686B'},
 1: {0: '#355423', 1: '#752F2E', 2: '#A19E54', 3: '#B0681C', 4: '#B5BCC9'}}